#**USING META'S "SEGMENT ANYTHING" MODEL TO CHANGE THE FLOORING OF THE HOUSE**

#**INSTALLING THE REQUIREMENTS**

In [6]:
#pip install segment-anything opencv-python pillow numpy


#**LOADING THE MODEL'S WEIGHTS**

In [7]:
import cv2
import numpy as np
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
from PIL import Image

# Load the image
image_path = "/content/house img.jpg"
image = cv2.imread(image_path)

# Convert the loaded image to RGB
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


# Loading SAM model's weight file
sam_checkpoint = "/content/sam_vit_b_01ec64.pth"
model_type = "vit_b"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mask_generator = SamAutomaticMaskGenerator(sam)


#**GENERATING THE MASK FOR THE IMAGE**

In [8]:
# Generate masks for the image
masks = mask_generator.generate(image)

# Assuming the floor is one of the masks
# Let's assume the largest mask corresponds to the floor
floor_mask = max(masks, key=lambda x: np.sum(x["segmentation"]))

# Create a binary mask for the floor
floor_binary_mask = floor_mask["segmentation"].astype(np.uint8) * 255

# Ensure the mask is binary
floor_binary_mask = cv2.threshold(floor_binary_mask, 127, 255, cv2.THRESH_BINARY)[1]


#**APPLYING A NEW FLOORING TEXTURE TO A SEGMENTED AREA IN AN IMAGE**

In [11]:
# Load and resize the new flooring texture
new_floor_texture = Image.open("/content/flooring.jpeg")
new_floor_texture = new_floor_texture.resize((image.shape[1], image.shape[0]))

# Convert to NumPy array
new_floor_texture = np.array(new_floor_texture)

# Ensure the texture is applied only where the mask is white
floor_area = cv2.bitwise_and(new_floor_texture, new_floor_texture, mask=floor_binary_mask)

# Invert the mask for the background
background_mask = cv2.bitwise_not(floor_binary_mask)
background = cv2.bitwise_and(image_rgb, image_rgb, mask=background_mask)

# Combine the floor area with the rest of the image
final_image = cv2.add(floor_area, background)

#**BLENDING NEW FLOORING TEXTURE WITH ORIGINAL IMAGE USING ALPHA MASKING**

In [12]:
# Create an alpha channel based on the mask
alpha = floor_binary_mask / 255.0

# Perform alpha blending manually
for c in range(0, 3):
    final_image[:, :, c] = (alpha * new_floor_texture[:, :, c] +
                            (1 - alpha) * image_rgb[:, :, c])

#**FINAL COLOR CONVERSION AND SAVING THE MODIFIED IMAGE**

In [13]:
# Convert back to BGR if needed
final_image_bgr = cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR)

# Save the final image
cv2.imwrite("modified_image.jpg", final_image_bgr)


True